In [1]:
#from __future__ import print_function, division
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

#import cv2 as cv

### Dataset и Dataloader

In [ ]:
from dataloader import LipReadingVideoDataset, LipReadingVideoDatasetZIP, collate_func

In [3]:
data_path = os.path.normpath('F:/lip_reading_data/LRS3/lrs3_trainval')
data_path_zip = os.path.normpath('F:/lip_reading_data/LRS3/lrs3_trainval.zip')

In [11]:
ds = LipReadingVideoDataset(data_path)

In [4]:
ds_zip = LipReadingVideoDatasetZIP(data_path_zip)

In [12]:
sample = ds[1]
vframes, text, txt_path, video_path = sample.values()

c:\program files\python39\lib\site-packages\torchvision\io\video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(


In [5]:
dl = torch.utils.data.DataLoader(ds_zip, batch_size=2, shuffle=True, collate_fn=collate_func)

In [6]:
for i, batch in enumerate(dl):
    print(batch.shape)
    if i > 0:
        break

c:\program files\python39\lib\site-packages\torchvision\io\video.py:158: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  warnings.warn(
c:\program files\python39\lib\site-packages\torch\utils\data\dataloader.py:557: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  data = self._dataset_fetcher.fetch(index)  # may raise StopIteration


torch.Size([2, 5, 128, 3, 224, 224])
torch.Size([2, 3, 128, 3, 224, 224])


### Показ видео

In [6]:
def video_show(sample):
    vframes = sample['vframes']
    face_df = sample['face']
    video_path = sample['video_path']
    
    cap = cv.VideoCapture(video_path)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            cv.imshow('frame', frame)
            # & 0xFF is required for a 64-bit system
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv.destroyAllWindows()

In [7]:
video_show(sample)

NameError: name 'cv' is not defined

### Network

In [7]:
batch.shape

torch.Size([2, 3, 128, 3, 224, 224])

In [26]:
class LipReadinNN(nn.Module):
    def __init__(self):
        super(self.__class__,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(224,224), padding=0)
        self.transf1 = nn.Transformer(d_model=9612, nhead=2, num_encoder_layers=2,
                                        num_decoder_layers=2, dim_feedforward=32)
        
        
    def forward(self, x):
        batch_size, frames_num, win_len, color, height, width = x.shape
        h = x.reshape(batch_size*frames_num*win_len, color, height, width)
        
        h = self.conv1(h)
        
        h = h.reshape(batch_size, frames_num, win_len, 16)
        
        h = self.transf1(h)
        
        return h

In [ ]:
my_network = LipReadinNN()

In [ ]:
output = my_network(batch)

In [99]:
output.shape

torch.Size([768, 8, 224, 224])

### Создание/считывание sentences

In [17]:
sentences = []

for dir_path, dir_names, file_names in os.walk(data_path):
    # перебрать файлы
    for file_name in file_names:
        if file_name.endswith('.txt'):
            txt_path = os.path.join(dir_path, file_name)
            txt_file = open(txt_path, 'r')
            
            _, text = txt_file.readline().split(':') # прочитанный текст
            text = text.strip()
            
            sentences.append(text)
            txt_file.close()

In [10]:
#file = open('sentences.txt', 'r')
#sentences = list(map(lambda line: line.strip(), file.readlines()))
#file.close()

In [11]:
se = sentences[:5]
se

['ONE DAY A YOUNG BOY COMES UPON THE SUNFLOWER WHILE VISITING THE GARDEN AND HE NOTICES HOW WEAK IT LOOKS',
 'JUST LIKE REACHING OUT TO THE SUNFLOWER BY PROVIDING SOMEONE WHO IS NEGLECTED ISOLATED OR FORGOTTEN',
 'THEY ARE INCREDIBLE YET SO OFTEN THEY ARE FORGOTTEN LACKING THE LOVE AND APPRECIATION THEY DESERVE',
 'I WILL HAVE HOPEFULLY MET MY GOAL',
 'THAT WAS COMPLETELY WRONG']

In [6]:
# может и не нужна т к тексты итак хорошие
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()

for i, sentence in enumerate(sentences):
    sentences[i] = tokenizer.tokenize(sentence)

In [12]:
from collections import Counter

token_counts = Counter(' '.join(sentences).split())

In [13]:
min_count = 2

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = []
for key, values in token_counts.items():
    if values >= min_count:
        tokens.append(key)

# Add a special tokens for unknown and empty words
UNK, PAD = "UNK", "PAD"
tokens = [UNK, PAD] + tokens

In [14]:
tokens

['UNK',
 'PAD',
 'ONE',
 'DAY',
 'A',
 'YOUNG',
 'BOY',
 'COMES',
 'UPON',
 'THE',
 'SUNFLOWER',
 'WHILE',
 'VISITING',
 'GARDEN',
 'AND',
 'HE',
 'HOW',
 'WEAK',
 'IT',
 'LOOKS',
 'JUST',
 'LIKE',
 'REACHING',
 'OUT',
 'TO',
 'BY',
 'PROVIDING',
 'SOMEONE',
 'WHO',
 'IS',
 'ISOLATED',
 'OR',
 'FORGOTTEN',
 'THEY',
 'ARE',
 'INCREDIBLE',
 'YET',
 'SO',
 'OFTEN',
 'LACKING',
 'LOVE',
 'APPRECIATION',
 'DESERVE',
 'I',
 'WILL',
 'HAVE',
 'HOPEFULLY',
 'MET',
 'MY',
 'GOAL',
 'THAT',
 'WAS',
 'COMPLETELY',
 'WRONG',
 'AS',
 'HAPPENED',
 'BECAUSE',
 'OF',
 'CONVENIENCE',
 'ACCESS',
 'ACTUALLY',
 'CHANGED',
 'BEHAVIOUR',
 'STARTED',
 'DOING',
 'FORTUNATE',
 'ENOUGH',
 'FIND',
 'JOB',
 'SUPPOSE',
 "YOU'D",
 'SAY',
 'THEN',
 'FROM',
 'BASICALLY',
 'BEEN',
 'IN',
 'CAREER',
 'EVER',
 'WHAT',
 'WE',
 'DO',
 'BETWEEN',
 'NOT',
 'BE',
 'BUT',
 'OUR',
 'LIVES',
 'REALLY',
 'MOST',
 'IMPORTANT',
 'US',
 'IMPACT',
 'OTHERS',
 'WORLD',
 'LEGACY',
 'LEAVE',
 'BEHIND',
 "I'D",
 'YOU',
 'GO',
 'AWAY',
 

In [21]:
token_to_id = {tok: i for i, tok in enumerate(tokens)}
id_to_token = {i: tok for tok, i in token_to_id.items()}

In [25]:
len(tokens)

9612